# Simple example of a recurrent SLDS

In [1]:
import os
import pickle

import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(12345)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
color_names = ["windows blue", "red", "amber", "faded green"]
colors = sns.xkcd_palette(color_names)
sns.set_style("white")
sns.set_context("talk")

from ssm.models import SLDS
from ssm.util import random_rotation

In [2]:
# Global parameters
T = 10000
K = 4
D_obs = 10
D_latent = 2

In [3]:
# Simulate the "nascar" data
def make_nascar_model():
    As = [random_rotation(D_latent, np.pi/24.),
      random_rotation(D_latent, np.pi/48.)]

    # Set the center points for each system
    centers = [np.array([+2.0, 0.]),
           np.array([-2.0, 0.])]
    bs = [-(A - np.eye(D_latent)).dot(center) for A, center in zip(As, centers)]

    # Add a "right" state
    As.append(np.eye(D_latent))
    bs.append(np.array([+0.1, 0.]))

    # Add a "right" state
    As.append(np.eye(D_latent))
    bs.append(np.array([-0.25, 0.]))

    # Construct multinomial regression to divvy up the space
    w1, b1 = np.array([+1.0, 0.0]), np.array([-2.0])   # x + b > 0 -> x > -b
    w2, b2 = np.array([-1.0, 0.0]), np.array([-2.0])   # -x + b > 0 -> x < b
    w3, b3 = np.array([0.0, +1.0]), np.array([0.0])    # y > 0
    w4, b4 = np.array([0.0, -1.0]), np.array([0.0])    # y < 0
    Rs = np.row_stack((100*w1, 100*w2, 10*w3,10*w4))
    r = np.concatenate((100*b1, 100*b2, 10*b3, 10*b4))
    
    true_rslds = SLDS(D_obs, K, D_latent, 
                      transitions="recurrent_only",
                      dynamics="gaussian",
                      emissions="gaussian",
                      single_subspace=True)
    true_rslds.dynamics.mu_init = np.array([0, 1])
    true_rslds.dynamics.inv_sigma_init = np.log(1e-4) * np.ones(2)
    true_rslds.dynamics.As = np.array(As)
    true_rslds.dynamics.bs = np.array(bs)
    true_rslds.dynamics.inv_sigmas = np.log(1e-4) * np.ones((K, D_latent))
    
    true_rslds.transitions.Rs = Rs
    true_rslds.transitions.r = r
    
    true_rslds.emissions.inv_etas = np.log(1e-2) * np.ones((1, D_obs))
    return true_rslds

# Sample from the model
true_rslds = make_nascar_model()
z, x, y = true_rslds.sample(T=T)

In [ ]:
# Fit a robust rSLDS with its default initialization
rslds = SLDS(D_obs, K, D_latent, 
             transitions="recurrent_only",
             dynamics="gaussian",
             emissions="gaussian",
             single_subspace=True)
elbos, (xhat, xvar) = rslds.fit(y, num_iters=1000, print_intvl=25)
zhat = rslds.most_likely_states(xhat, y)

Initializing with an ARHMM using 25 steps of EM.
Done
Iteration 0.  ELBO: 61736.9
Iteration 25.  ELBO: 64720.4
Iteration 50.  ELBO: 66149.5
Iteration 75.  ELBO: 67761.0
Iteration 100.  ELBO: 68491.5
Iteration 125.  ELBO: 69426.1
Iteration 150.  ELBO: 69845.8
Iteration 175.  ELBO: 70576.2
Iteration 200.  ELBO: 70654.6
Iteration 225.  ELBO: 71147.7
Iteration 250.  ELBO: 71595.0
Iteration 275.  ELBO: 71815.3
Iteration 300.  ELBO: 71959.1
Iteration 325.  ELBO: 71942.5
Iteration 350.  ELBO: 71885.0
Iteration 375.  ELBO: 71878.4
Iteration 400.  ELBO: 72403.1
Iteration 425.  ELBO: 72490.5
Iteration 450.  ELBO: 72814.4
Iteration 475.  ELBO: 72855.0
Iteration 500.  ELBO: 72981.8
Iteration 525.  ELBO: 73127.1
Iteration 550.  ELBO: 73088.6
Iteration 575.  ELBO: 73442.8
Iteration 600.  ELBO: 73245.8


In [ ]:
# Plot some results
plt.figure()
plt.plot(elbos)
plt.xlabel("Iteration")
plt.ylabel("ELBO")

In [ ]:
# Helper functions for plotting results
def plot_trajectory(z, x, ax=None, ls="-"):
    zcps = np.concatenate(([0], np.where(np.diff(z))[0] + 1, [z.size]))
    if ax is None:
        fig = plt.figure(figsize=(4, 4))
        ax = fig.gca()
    for start, stop in zip(zcps[:-1], zcps[1:]):
        ax.plot(x[start:stop + 1, 0],
                x[start:stop + 1, 1],
                lw=1, ls=ls,
                color=colors[z[start] % len(colors)],
                alpha=1.0)

    return ax


def plot_most_likely_dynamics(model,
    xlim=(-4, 4), ylim=(-3, 3), nxpts=20, nypts=20,
    alpha=0.8, ax=None, figsize=(3, 3)):
    
    K = model.K
    assert model.D == 2
    x = np.linspace(*xlim, nxpts)
    y = np.linspace(*ylim, nypts)
    X, Y = np.meshgrid(x, y)
    xy = np.column_stack((X.ravel(), Y.ravel()))

    # Get the probability of each state at each xy location
    z = np.argmax(xy.dot(model.transitions.Rs.T) + model.transitions.r, axis=1)

    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111)

    for k, (A, b) in enumerate(zip(model.dynamics.As, model.dynamics.bs)):
        dxydt_m = xy.dot(A.T) + b - xy

        zk = z == k
        if zk.sum(0) > 0:
            ax.quiver(xy[zk, 0], xy[zk, 1],
                      dxydt_m[zk, 0], dxydt_m[zk, 1],
                      color=colors[k % len(colors)], alpha=alpha)

    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')

    plt.tight_layout()

    return ax

In [ ]:
plt.figure(figsize=(8,4))
ax1 = plt.subplot(121)
plot_trajectory(z, x, ax=ax1)
plt.title("True")
ax2 = plt.subplot(122)
plot_trajectory(zhat, xhat, ax=ax2)
plt.title("Inferred")

In [ ]:
plt.figure(figsize=(6,4))
ax = plt.subplot(111)
lim = abs(x).max(axis=0) + 1
plot_most_likely_dynamics(true_rslds, xlim=(-lim[0], lim[0]), ylim=(-lim[1], lim[1]), ax=ax)
plt.title("True Dynamics")

plt.figure(figsize=(6,4))
ax = plt.subplot(111)
lim = abs(xhat).max(axis=0) + 1
plot_most_likely_dynamics(rslds, xlim=(-lim[0], lim[0]), ylim=(-lim[1], lim[1]), ax=ax)
plt.title("Inferred Dynamics")